In [1]:
import glob
import os
import cv2
import numpy as np
from PIL import Image
import torch 
from PIL import Image
from pathlib import Path
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import numpy as np
from torchvision import transforms
import os
import cv2
import matplotlib.pyplot as plt
import shutil

/home/wej36how/.conda/envs/dmt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
source = "/scratch/wej36how/Datasets/NWRD/val"
dest = "/scratch/wej36how/Datasets/NWRDProcessed/val"
patch_size = 224
rust_threshold = 150
max_number_of_images_per_group = 12

This snippet will make patches of the images in the destination/patches directory.

In [ ]:
patches_path = os.path.join(dest, "patches")
images_dir = os.path.join(patches_path, "images")
masks_dir = os.path.join(patches_path, "masks")

In [3]:
masks_paths = glob.glob(f'{os.path.join(source, "masks", "*")}')
images_paths = glob.glob(f'{os.path.join(source, "images", "*")}')
images_paths.sort()
masks_paths.sort()


os.makedirs(patches_path)
os.makedirs(images_dir)
os.makedirs(masks_dir)

def create_patches(fname):
    x = 0
    y = 0
    patches = []
    img = cv2.imread(fname)
    print("image shape:",img.shape)
    p_num = 0
    while (y + patch_size < img.shape[0]):
        
        if (x + patch_size > img.shape[1]):
            x = 0
            y += patch_size
        if y + patch_size <= img.shape[0] and x + patch_size <= img.shape[1]:
            patches.append([x, y])
        x += patch_size
    print("total patches: ", len(patches))
    return patches

total_count = 0
for u in images_paths:
    print(u)
    patches = create_patches(u)
    bgr = cv2.imread(u)
    image_name = u.split('/')[-1].split('.')[0]
    total_count += len(patches)

    for count, P in enumerate(patches):
        cv2.imwrite(os.path.join(images_dir,f"{image_name}_{count}.png"), bgr[P[1]:P[1]+patch_size,P[0]:P[0]+patch_size])
            
print("total image count:", total_count)

total_count = 0
for u in masks_paths:
    print(u)
    patches = create_patches(u)
    bgr = cv2.imread(u)
    image_name = u.split('/')[-1].split('.')[0]

    total_count += len(patches)

    for count, P in enumerate(patches):
        cv2.imwrite(os.path.join(masks_dir,f"{image_name}_{count}.png"), bgr[P[1]:P[1]+patch_size,P[0]:P[0]+patch_size])
            
print("total masks count:", total_count)

/scratch/wej36how/Datasets/NWRD/train/images/10.jpg
image shape: (4000, 6016, 3)
total patches:  442
/scratch/wej36how/Datasets/NWRD/train/images/100.jpg
image shape: (4608, 3456, 3)
total patches:  300
/scratch/wej36how/Datasets/NWRD/train/images/101.jpg
image shape: (3456, 3612, 3)
total patches:  240
/scratch/wej36how/Datasets/NWRD/train/images/102.jpg
image shape: (2984, 4248, 3)
total patches:  234
/scratch/wej36how/Datasets/NWRD/train/images/103.jpg
image shape: (3584, 3456, 3)
total patches:  226
/scratch/wej36how/Datasets/NWRD/train/images/104.jpg
image shape: (3456, 4608, 3)
total patches:  300
/scratch/wej36how/Datasets/NWRD/train/images/109.jpg
image shape: (4608, 3456, 3)
total patches:  300
/scratch/wej36how/Datasets/NWRD/train/images/11.jpg
image shape: (4000, 6016, 3)
total patches:  442
/scratch/wej36how/Datasets/NWRD/train/images/110.jpg
image shape: (4600, 2536, 3)
total patches:  220
/scratch/wej36how/Datasets/NWRD/train/images/111.jpg
image shape: (2909, 4608, 3)
to

This will saperate the rust and non rust patches and put them in and put them in directory destination/RustNonRustSplit

In [4]:
destination = os.path.join(dest, "RustNonRustSplit")
root = patches_path

os.makedirs(destination)
os.makedirs(os.path.join(destination,"non_rust","images"))
os.makedirs(os.path.join(destination,"non_rust","masks"))
os.makedirs(os.path.join(destination,"rust","images"))
os.makedirs(os.path.join(destination,"rust","masks"))

masks_path = os.path.join(root, "masks", "*.png")
masks_paths = glob.glob(masks_path)
minimum=1000
min_patch=0
rust_count=0
non_rust_count=0

for mask_path in masks_paths:
    patch_name = mask_path.split("/")[-1].split(".")[0]
    
    patch_mask = cv2.imread(mask_path, 0)
    patch_img = cv2.imread(os.path.join(root, "images",patch_name+".png"))

    condition = (patch_mask > 150)
    count = np.sum(condition)
        
    if count<=rust_threshold:
            cv2.imwrite(os.path.join(destination,"non_rust","images",f"{patch_name}.png"), patch_img)
            cv2.imwrite(os.path.join(destination,"non_rust","masks",f"{patch_name}.png"), patch_mask)
            non_rust_count+=1
    else:
            if (count<=minimum):
                   minimum=count
                   min_patch = patch_name
            cv2.imwrite(os.path.join(destination,"rust","images",f"{patch_name}.png"), patch_img)
            cv2.imwrite(os.path.join(destination,"rust","masks",f"{patch_name}.png"), patch_mask)
            rust_count+=1

print("minimum rust patch:",min_patch)
print("minimum rust patch white pixels:",minimum)
print("rust count=", rust_count)
print("non rust count=", non_rust_count)

Run the next two code snippets for training only. The following code will augment the images in the destination/RustNonRustSplit/images and destination/RustNonRustSplit/masks folder. 

In [ ]:
#flip images horizontally
def flip_images_hor(input_image):
    # Iterate over the images in the input directory
    transform_hflip = T.RandomHorizontalFlip(p=1.0)  # Set probability to 1.0 to always flip
    return transform_hflip(input_image)

#flip images vertically
def flip_images_ver(input_image):
    # Iterate over the images in the input directory
    transform_vflip = T.RandomVerticalFlip(p=1.0)  # Set probability to 1.0 to always flip
    return transform_vflip(input_image) 
  
def shear_vertical(input_image, shear_factor=45):
    # Apply vertical shear
    sheared_image = TF.affine(input_image, angle=0, translate=(0, 0), scale=1, shear=(0, shear_factor))
    return sheared_image

def shear_horizontal(input_image, shear_factor=45):  # Increased shear for testing
    sheared_image = TF.affine(input_image, angle=0, translate=(0, 0), scale=1, shear=(shear_factor, 0))
    return sheared_image

def rotate_images(input_image, angle=45):
    # Convert PIL Image to NumPy array
    input_array = np.array(input_image)
    # Rotate the image
    height, width = input_array.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    rotated_array = cv2.warpAffine(input_array, rotation_matrix, (width, height))
    # Convert NumPy array back to PIL Image
    rotated_image = Image.fromarray(rotated_array)
    return rotated_image

def dark(input_image,gamma):
    dark_image= TF.adjust_gamma(input_image, gamma)
    return dark_image

def augment_image(img_path):

  # Apply the transformations
  
    #orig_image
    orig_img = Image.open(Path(img_path))
    
    #flip images
    img_hflipped = flip_images_hor(orig_img)
    img_vflipped = flip_images_ver(orig_img)
    
    
    #shear images
    hor_shear = shear_horizontal(orig_img)
    ver_shear = shear_vertical(orig_img)
    
    #dark
    img_dark = dark(img_hflipped, 2)
    img_rot = rotate_images(orig_img, angle=45)
    
    return [img_dark,img_hflipped,img_vflipped,hor_shear,ver_shear, img_rot]

def creating_file_with_augmented_images(file_path_master_dataset, file_path_augmented_images):
    master_dataset_folder = file_path_master_dataset
    files_in_master_dataset = os.listdir(file_path_master_dataset)
    augmented_images_folder = file_path_augmented_images
    
    for image_name in files_in_master_dataset:
        image_path = os.path.join(master_dataset_folder, image_name)
        required_images = augment_image(image_path)  # Assuming augment_image is defined elsewhere
        i = 0
        for augmented_image in required_images:
            # Convert RGBA to RGB if necessary
            if augmented_image.mode == 'RGBA':
                augmented_image = augmented_image.convert('RGB')
                
            # Save as png
            augmented_image_path = os.path.join(augmented_images_folder, f"aug{i}_{image_name}")
            augmented_image.save(augmented_image_path, format='png')
            i += 1

master_dataset = os.path.join(destination,"rust","images")
augmented_dataset = os.path.join(destination,"rust","images")
creating_file_with_augmented_images(master_dataset,augmented_dataset)

master_dataset = os.path.join(destination,"rust","masks")
augmented_dataset = os.path.join(destination,"rust","masks")
creating_file_with_augmented_images(master_dataset,augmented_dataset)

Run next snippet only for training dataset. To remove patches that have their rust removed becuase of their augmentations

In [ ]:
root = os.path.join(destination,"rust")

non_rust_images_dir = os.path.join(destination,"non_rust","images")
non_rust_masks_dir = os.path.join(destination,"non_rust","masks")

masks_path = os.path.join(root, "masks", "*.png")
masks_paths = glob.glob(masks_path)
minimum=1000
min_patch=0
rust_count=0
non_rust_count=0

for mask_path in masks_paths:
    patch_name = mask_path.split("/")[-1].split(".")[0]
    
    patch_mask = cv2.imread(mask_path, 0)
    patch_img = cv2.imread(os.path.join(root, "images",patch_name+".png"))

    condition = (patch_mask > 150)
    count = np.sum(condition)
        
    if count<=rust_threshold:
            os.remove(mask_path)
            os.remove(os.path.join(root, "images",patch_name+".png"))

            cv2.imwrite(os.path.join(non_rust_images_dir,f"{patch_name}.png"), patch_img)
            cv2.imwrite(os.path.join(non_rust_masks_dir,f"{patch_name}.png"), patch_mask)
            non_rust_count+=1
    else:
            if (count<=minimum):
                   minimum=count
                   min_patch = patch_name
        #     cv2.imwrite(os.path.join(destination,"rust","images",f"{patch_name}.png"), patch_img)
        #     cv2.imwrite(os.path.join(destination,"rust","masks",f"{patch_name}.png"), patch_mask)
            rust_count+=1

print("minimum rust patch:",min_patch)
print("minimum rust patch white pixels:",minimum)
print("rust count=", rust_count)
print("non rust count=", non_rust_count)

minimum rust patch: 0
minimum rust patch white pixels: 1000
rust count= 0
non rust count= 0


Create a dataset for classification model

In [ ]:
rust_images_dir = os.path.join(destination,"rust","images")
non_rust_images_dir = os.path.join(destination,"non_rust","images")

rustClassificationDir = os.path.join(dest, "calssification", "rust")
nonRustClassificationDir = os.path.join(dest, "calssification", "non_rust")
os.makedirs(rustClassificationDir, exist_ok=True)
os.makedirs(nonRustClassificationDir, exist_ok=True)

shutil.copytree(rust_images_dir,rustClassificationDir, dirs_exist_ok=True)
shutil.copytree(non_rust_images_dir,nonRustClassificationDir, dirs_exist_ok=True)


'C:\\Users\\hasee\\Desktop\\Germany_2024\\Dataset\\NWRDFprocessed\\train\\calssification\\non_rust'

Run the next code snippet for training dataset only. It deletes non-rust patches to match rust patches in the classification folder only.

In [ ]:
import os
import glob

def delete_extra_images(directory, target_count):
    # Get a list of all image files in the directory
    image_files = glob.glob(os.path.join(directory, '*.JPG')) + glob.glob(os.path.join(directory, '*.jpeg')) + glob.glob(os.path.join(directory, '*.png'))
    
    # Check if the number of images exceeds the target count
    if len(image_files) > target_count:
        # Calculate the number of images to delete
        num_to_delete = len(image_files) - target_count
        # Sort the images by modification time (oldest first)
        image_files.sort(key=os.path.getmtime)
        # Delete the extra images
        for i in range(num_to_delete):
            os.remove(image_files[i])
        print(f"{num_to_delete} images deleted.")
    elif len(image_files) < target_count:
        print("Warning: Number of images in directory is less than the target count.")

if len(os.listdir(rustClassificationDir))< len(os.listdir(nonRustClassificationDir)):
    delete_extra_images(nonRustClassificationDir, len(os.listdir(rustClassificationDir)))
else:
    delete_extra_images(rustClassificationDir, len(os.listdir(nonRustClassificationDir)))


The following code creates a coslaiency style structure for co-saliency models training

In [ ]:
rust_dir = os.path.join(destination,"rust")
rustCosaliencynDir = os.path.join(dest, "cosaliency")
shutil.copytree(rust_dir,rustCosaliencynDir, dirs_exist_ok=True)

# Function to split images into folders based on image number
def split_images_into_folders(source_dir, destination_dir):
    # Create destination directory if it doesn't exist
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)
    # Iterate through files in the source directory
    for filename in os.listdir(source_dir):
        if filename.endswith('.png'):
            image_no = filename.split('_')[0]  # Extract image number from filename
            if not image_no.isdigit():
                image_no = filename.split('_')[1]
            destination_subdir = os.path.join(destination_dir, image_no)
            # Create subdirectory if it doesn't exist
            if not os.path.exists(destination_subdir):
                os.makedirs(destination_subdir)
            # Move the image file to the respective subdirectory
            shutil.move(os.path.join(source_dir, filename), destination_subdir)

def organize_images(main_directory):
    # Ensure the main directory exists
    if not os.path.exists(main_directory):
        print(f"The specified main directory '{main_directory}' does not exist.")
        return

    # Get a list of subdirectories in the main directory
    subdirectories = [d for d in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, d))]

    # Process each subdirectory
    for subdir in subdirectories:
        subdir_path = os.path.join(main_directory, subdir)

        # Get a list of images in the subdirectory
        images = [f for f in os.listdir(subdir_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))]
        # Determine the number of images per subdirectory
        images_per_subdir = 12
        num_subdirectories = len(images) // images_per_subdir
        n=0
        # Create additional subdirectories if needed
        for i in range(num_subdirectories - 1):
            new_subdir_name = f"{subdir}_part{i + 1}"
            new_subdir_path = os.path.join(main_directory, new_subdir_name)

            # Create the new subdirectory
            os.makedirs(new_subdir_path)

            # Move images to the new subdirectory
            for j in range(images_per_subdir):
                old_image_path = os.path.join(subdir_path, images[n])
                new_image_path = os.path.join(new_subdir_path, images[n])
                shutil.move(old_image_path, new_image_path)
                n+=1

source_directory = os.path.join(dest, "cosaliency", "images")
destination_directory = os.path.join(dest, "cosaliency", "images")
split_images_into_folders(source_directory, destination_directory)
organize_images(destination_directory)

source_directory = os.path.join(dest, "cosaliency", "masks")
destination_directory = os.path.join(dest, "cosaliency", "masks")
split_images_into_folders(source_directory, destination_directory)
organize_images(destination_directory)